### 1. 滑动窗口最大值
双端队列：保存窗口内最大值，头为最大值，尾cur进入时pop掉<=cur的所有数。左侧缩窗口保证窗口内不过期。
    窗口内所有值，最多进一次、出一次。更新总代价O(N)，平均每次最大值查询O(1)

#### 1.1 固定大小W的窗口，依次划过arr。返回每次划出状况的最大值
* arr = [4,3,5,4,3,3,6,7]，W=3
* 返回:  [5,5,5,4,6,7]

In [1]:
def right(arr, W):
    if not arr or len(arr) < W or W < 1:
        return None
    N = len(arr)
    res = [None] * (N-W+1)
    index = 0
    L = 0
    R = W - 1
    while R <= N - 1:
        m = arr[L]
        for i in range(L+1, R+1):
            m = max(m, arr[i])
        res[index] = m
        index += 1
        L += 1
        R += 1
    return res
        

In [8]:
from collections import deque
def getMaxWindow(arr, W):
    if not arr or len(arr) < W or W < 1:
        return None
    #qmax放下标
    qmax = deque()
    N = len(arr)
    res = [None] * (N-W+1)
    index = 0
    #R代表窗口右边界
    for R in range(0,N):
        while qmax and arr[qmax[-1]] <= arr[R]:
            qmax.pop()
        qmax.append(R)
        if qmax[0] == R - W: #弹出过期左
            qmax.popleft()
        if R >= W - 1:       #判断是否够W个数
            res[index] = arr[qmax[0]]
            index += 1
    return res

In [9]:
testTime = 10000
maxValue = 100
maxLen = 100
maxW = 30
succeed=True
for _ in range(testTime):
    value = random.randint(0, maxValue)
    length = random.randint(0, maxLen)
    W = random.randint(1, maxW)
    arr = generateRandomArr(length, value)
    ans1 = right(arr, W)
    ans2 = getMaxWindow(arr, W)
    if ans1 != ans2:
        print(ans1, ans2)
        print("something wrong.")
        succeed = False
        break
if succeed:
    print("succeed.")

succeed.


In [3]:
import random
def generateRandomArr(maxLen, maxValue):
    arr = list()
    for i in range(maxLen):
        arr.append(random.randint(-maxValue, maxValue))
    return arr

#### 1.2 最大值-最小值达标窗口
给定一个arr，和一个整数num，问有多个子数组满足：最大值-最小值<= num

In [ ]:
'''
1.已知L...R的数组达标：L...R的任意子数组都达标
2.已知L...R的数组不达标，那么L往左扩或者R往右扩，都不达标。
求解思路：
准备一个qmax；一个qmin。
从0作为L开始把右边界往右扩，每次更新qmax、qmin，右扩到R不达标停止。此时以0开头的子数组达标数量为R-L。
然后左侧左滑一个，L=1，右侧从R开始继续看能不能右扩。重复过程一直到L到N-1或
'''
def sizes(arr, num):
    if not arr or num <0:
        return 0
    N = len(arr)
    count = 0
    maxWindow = deque()
    minWindow = deque()
    R = 0
    for L in range(N):
        #窗口[L,R), 当L=R，说明窗口里没数
        while R < N:
            while maxWindow and arr[maxWindow[-1]] <= arr[R]:
                maxWindow.pop()
            maxWindow.append(R)
            while minWindow and arr[minWindow[-1]] >= arr[R]:
                minWindow.pop()
            minWindow.append(R)
            #初次不达标，停
            if arr[maxWindow[-1]] - arr[minWindow[-1]] > num:
                break
            else:
                R += 1
        count += R - L
        #L即将+1，L过期
        if maxWindow[0] == L:
            maxWindow.popleft()
        if minWindow[0] == L:
            minWindow.popleft()
    return count

#### 1.3 加油站良好出发问题
给定两个数组gas、cost，gas[i]、cost[i]代表 i号加油站有gas[i]的油，需要cost[i]的油才能到i+1号加油站。问从哪个加油站开始可以转完一圈，返回每个加油站是否可以转完一圈的arr

In [49]:
'''
思路：
1. 先合并gas、cost => arr，每次途径arr[i]时，加上arr[i]现有的油不能掉到0以下。 等同于 arr[i]的累加和不能出现0
2. 准备两倍长arr，计算累和
例如：arr[-2,-3,4,-2,6,-1]
累加和。 [-2,-5,-1,-3,3,2,0,-3,1,-1, 5,4]
相当于下标 0, 1, 2, 3,4,5,6, 7,8, 9,10,11] 为的是以x(0~5)作为出发点时，累加和都通过sum(x～x+len(arr)-1) - (x-1)可以加出来。
当 以 1节点出发，那么1~6 减去 0 即为累加和.
3. 准备长度为len(arr)的窗口，窗口内添加累加和，窗口内最小值(需要减去前一个值还原)<0，则L不是有效出发点/
'''
def getNextOne(arr, index):
    return arr[index % (len(arr))]
def gasCycle(gas, cost):
    if not gas or not cost or len(gas) != len(cost):
        return None
    N = len(gas)
    arr = [gas[i] - cost[i] for i in range(N)]
    arrSum = [None] * (N * 2 - 1)
    arrSum[0] = arr[0]
    for i in range(1, len(arrSum)):
        arrSum[i] = arrSum[i-1] + getNextOne(arr, i)
    L = 0
    R = 0
    ans = [None] * N
    queue = deque()
    while L <= N - 1:
        while queue and arrSum[queue[-1]] >= arrSum[R]:
            queue.pop()
        queue.append(R)
        
        if R - L + 1 == N:
#             print(queue[0])
            minV = arrSum[queue[0]]
            if L >= 1:
                minV -= arrSum[L-1]
            if minV < 0:
                ans[L] = False
            else:
                ans[L] = True
            if queue[0] == L:
                queue.popleft()
            L += 1 
        R += 1
    return ans


In [50]:
gas = [1, 4, 5, 2, 9, 2]
cost= [3, 7, 1, 4, 3, 3]
#[-2,-3,4,-2,6,-1]
gasCycle(gas, cost)

[False, False, True, False, True, False]

#### arr货币，返回组成aim的最少货币数


In [14]:
#最小值时，不能根据位置依赖计算出， 因为不知道某个数出去后的最小值是多少
#使用窗口内最小值结构，位置进出，返回最小值；每次pk、取最小值时，要补齐当前数(x)和尾巴(y)差几张(尾巴实际=y + (x-y)/面值)

In [52]:
import sys
sys.maxsize

9223372036854775807

In [136]:
import random
def generateRandomPosArr(maxLen, maxValue):
    arr = list()
    for i in range(maxLen):
        arr.append(random.randint(1, maxValue))
    return arr

In [92]:
def minMoneyPre(arr, aim):
    if not arr or aim <= 0:
        return -1
    count = dict()
    for i in arr:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    money = sorted(count.keys())
    ans = process(money, count, aim, 0)
    return ans

def process(money, count, rest, index):
    if index == len(money):
        return -1 if rest != 0 else 0
    if rest == 0:
        return 0
    ans = -1
    for i in range(count[money[index]]+1):
#         print(i, money[index])
        if rest - i * money[index] >= 0:
            nxt = process(money, count, rest - i * money[index], index+1)
            if nxt != -1:
                nxt += i
            else:
                continue
            if ans == -1:
                ans = nxt
            else:
                ans = min(ans, nxt)
    return ans

def dp(arr, aim):
    if not arr or aim <= 0:
        return -1
    count = dict()
    for i in arr:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    money = sorted(count.keys())
    M = len(money)
    dp = [[-1] * (aim+1) for _ in range(M+1)]
    for i in range(M+1):
        dp[i][0] = 0
    for i in range(M-1, -1, -1):
        for rest in range(1, aim+1):
            dp[i][rest] = dp[i+1][rest]
            for num in range(1, count[money[i]]+1):
                if rest - num * money[i] >= 0:
                    nxt = dp[i+1][rest - num * money[i]]
                    if nxt != -1:
                        nxt += num
                    else:
                        continue
                    if dp[i][rest] == -1:
                        dp[i][rest] = nxt
                    else:
                        dp[i][rest] = min(dp[i][rest], nxt)
    return dp[0][aim]

In [279]:
def dp2(arr, aim):
    if aim == 0:
        return 0
    count = dict()
    for i in arr:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    money = sorted(count.keys())
    N = len(money)
    dp = [[-1] * (aim+1) for _ in range(N+1)]
    dp[N][0] = 0
    for index in range(N-1,-1,-1):
        for start in range(0, min(aim+1, money[index])):
            queue = deque()
            queue.append(start)
            dp[index][start] = dp[index+1][start]
            step = money[index]
            window = money[index] * (count[money[index]]+1)
            R = start + step
            while R <= aim:
                while queue and (dp[index+1][queue[-1]] == -1 or \
                                 (dp[index+1][R] != -1 and dp[index+1][queue[-1]] + (R-queue[-1]) / step >= dp[index+1][R])
                                ):
                    queue.pop()
                queue.append(R)
                if R - window == queue[0]:
                    queue.popleft()
                dp[index][R] = -1 if dp[index+1][queue[0]] == -1 else dp[index+1][queue[0]] + (R-queue[0]) / step
                R += step
    return dp[0][aim]
                

In [280]:
def dp3(arr, aim):
    if aim == 0:
        return 0
    count = dict()
    for i in arr:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    money = sorted(count.keys())
    N = len(money)
    dp = [[-1] * (aim+1) for _ in range(N+1)]
    dp[N][0] = 0
    for j in range(1, aim+1):
        dp[N][j] = sys.maxsize;
    for i in range(N-1, -1, -1):
        for mod in range(0, min(aim+1, money[i])):
            w = deque()
            w.append(mod)
            dp[i][mod] = dp[i + 1][mod]
            for r in range(mod+money[i], aim+1, money[i]):
                while w and ((dp[i+1][w[-1]] == sys.maxsize) or \
                (dp[i+1][w[-1]] + compensate(w[-1], r, money[i]) >= dp[i+1][r])):
                    w.pop()
                w.append(r)
                overdue = r - money[i] * (count[money[i]] + 1)
                if w[0] == overdue:
                    w.popleft()
#                 if dp[i + 1][w[0]] == -1:
#                     dp[i][r] = -1
#                 else:
#                     dp[i][r] = dp[i + 1][w[0]] + compensate(w[0], r, money[i])
                dp[i][r] = dp[i + 1][w[0]] + compensate(w[0], r, money[i])
    if dp[0][aim] >= sys.maxsize:
        return -1
    else:
        return dp[0][aim]

def compensate(pre, cur, coin):
    return (cur - pre) / coin

In [282]:
testTime = 10000
maxValue = 100
maxLen = 30
maxAim = 100
succeed=True
for _ in range(testTime):
    value = random.randint(1, maxValue)
    length = random.randint(0, maxLen)
    aim = random.randint(1, maxAim)
    arr = generateRandomPosArr(length, value)
    ans1 = dp2(arr, aim)
    ans2 = dp3(arr, aim)
    if ans1 != ans2:
        print(ans1, ans2)
        print("something wrong.")
        succeed = False
        break
if succeed:
    print("succeed.")

succeed.


In [276]:
arr, aim

([2, 6, 9], 6)

In [88]:
minMoneyPre(arr, maxAim)

2

In [75]:
maxAim

17